In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import keras as k
from keras.utils import to_categorical
from keras.layers import *
import pickle

In [0]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving student-mat.csv to student-mat.csv
User uploaded file "student-mat.csv" with length 56993 bytes


In [0]:
data=pd.read_csv("student-mat.csv",sep=";")

In [0]:
data.loc[0:5,"age"]

0    18
1    17
2    15
3    15
4    16
5    16
Name: age, dtype: int64

In [0]:
data_x=data.iloc[:,1:32]
y=data.iloc[:,32:]

In [0]:
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  
    for col, col_data in X.iteritems():
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
            

        if col_data.dtype == object: 
            col_data = pd.get_dummies(col_data, prefix=col) 
        
        outX = outX.join(col_data)  

    return outX

x=preprocess_features(data_x)

In [0]:
x.columns

Index(['sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3',
       'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home',
       'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher',
       'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services',
       'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other',
       'reason_reputation', 'guardian_father', 'guardian_mother',
       'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup',
       'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet',
       'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health',
       'absences', 'G1', 'G2'],
      dtype='object')

In [0]:
data_x.columns

Index(['sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob',
       'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures',
       'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
       'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc',
       'health', 'absences', 'G1', 'G2'],
      dtype='object')

In [61]:
data_x

,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,...,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2
0,F,18,U,GT3,A,4,4,at_home,teacher,course,...,no,4,3,4,1,1,3,6,5,6
1,F,17,U,GT3,T,1,1,at_home,other,course,...,no,5,3,3,1,1,3,4,5,5
2,F,15,U,LE3,T,1,1,at_home,other,other,...,no,4,3,2,2,3,3,10,7,8
3,F,15,U,GT3,T,4,2,health,services,home,...,yes,3,2,2,1,1,5,2,15,14
4,F,16,U,GT3,T,3,3,other,other,home,...,no,4,3,2,1,2,5,4,6,10
5,M,16,U,LE3,T,4,3,services,other,reputation,...,no,5,4,2,1,2,5,10,15,15
6,M,16,U,LE3,T,2,2,other,other,home,...,no,4,4,4,1,1,3,0,12,12
7,F,17,U,GT3,A,4,4,other,teacher,home,...,no,4,1,4,1,1,1,6,6,5
8,M,15,U,LE3,A,3,2,services,other,home,...,no,4,2,2,1,1,1,0,16,18
9,M,15,U,GT3,T,3,4,other,other,home,...,no,5,5,1,1,1,5,0,14,15


In [0]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=3,shuffle=True)

In [0]:
x_tr=x_train.as_matrix()
y_tr=y_train.as_matrix()
x_te=x_test.as_matrix()
y_te=y_test.as_matrix()

In [0]:
y_train=to_categorical(y_tr)

In [0]:
y_test=np.zeros((len(y_te),21))
for i in range(len(y_te)):
  y_test[i][y_te[i]]=1

In [0]:
model=k.Sequential()
model.add(Dense(48,input_shape=(48,),activation="relu"))
"""model.add(Dropout(0.5))
model.add(Dense(40,activation="relu"))"""
model.add(Dropout(0.5))
model.add(Dense(21,activation='softmax'))
model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 48)                2352      
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                1029      
Total params: 3,381
Trainable params: 3,381
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.fit(x_tr, y_train,validation_data=(x_te,y_test), epochs=2000)

Train on 316 samples, validate on 79 samples
Epoch 1/2000
316/316 [==============================] - 0s 72us/step - loss: 0.6652 - acc: 0.7563 - val_loss: 2.4332 - val_acc: 0.4051
Epoch 2/2000
316/316 [==============================] - 0s 54us/step - loss: 0.7160 - acc: 0.7089 - val_loss: 2.4477 - val_acc: 0.4177
Epoch 3/2000
316/316 [==============================] - 0s 53us/step - loss: 0.6980 - acc: 0.7215 - val_loss: 2.4086 - val_acc: 0.4557
Epoch 4/2000
316/316 [==============================] - 0s 53us/step - loss: 0.6944 - acc: 0.7152 - val_loss: 2.3889 - val_acc: 0.4051
Epoch 5/2000
316/316 [==============================] - 0s 58us/step - loss: 0.7768 - acc: 0.6646 - val_loss: 2.4505 - val_acc: 0.4177
Epoch 6/2000
316/316 [==============================] - 0s 63us/step - loss: 0.6469 - acc: 0.7658 - val_loss: 2.3629 - val_acc: 0.4304
Epoch 7/2000
316/316 [==============================] - 0s 61us/step - loss: 0.6944 - acc: 0.7437 - val_loss: 2.4031 - val_acc: 0.4177
Epoch 8/20

In [69]:
np.argmax(model.predict(x_te[6].reshape(1,48)))

10

In [72]:
x_te[6]

array([ 1,  0, 17,  0,  1,  1,  0,  0,  1,  2,  2,  0,  0,  1,  0,  0,  0,
        0,  1,  0,  0,  1,  0,  0,  0,  0,  1,  0,  1,  2,  0,  0,  1,  0,
        0,  0,  1,  1,  0,  5,  4,  5,  1,  2,  5,  4, 10,  9])

In [71]:
_test.iloc[6]

sex_F                 1
sex_M                 0
age                  17
address_R             0
address_U             1
famsize_GT3           1
famsize_LE3           0
Pstatus_A             0
Pstatus_T             1
Medu                  2
Fedu                  2
Mjob_at_home          0
Mjob_health           0
Mjob_other            1
Mjob_services         0
Mjob_teacher          0
Fjob_at_home          0
Fjob_health           0
Fjob_other            1
Fjob_services         0
Fjob_teacher          0
reason_course         1
reason_home           0
reason_other          0
reason_reputation     0
guardian_father       0
guardian_mother       1
guardian_other        0
traveltime            1
studytime             2
failures              0
schoolsup             0
famsup                1
paid                  0
activities            0
nursery               0
higher                1
internet              1
romantic              0
famrel                5
freetime              4
goout           

In [0]:
y_te[6]

array([11])

In [0]:
c=0
for i in range(len(y_te)):
  p=np.argmax(model.predict(x_te[i].reshape(1,48)))
  if abs(y_te[i]-p)>1:
    c=c+1

In [0]:
len(y_te)

SyntaxError: ignored

In [0]:
c

26

In [0]:
pickle.dump(model,open("LSD.p","wb"))